# Test to Ensure GPU Functional and Available

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn import datasets
import pandas as pd
import numpy as np
import datetime
print(tf.__version__)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

__DD:__ Check to see if CUDA GPU is available:

In [ ]:
tf.test.is_gpu_available(
    cuda_only=True, min_cuda_compute_capability=None
)

__DD:__ Try new function based on warnings:

In [ ]:
tf.config.list_physical_devices()

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

__DD:__ I was wondering how to make sure GPU does the processing rather than CPU. I found the following excerpt from Tensorflow documentation:  

"If a TensorFlow operation has both CPU and GPU implementations, by default the GPU devices will be given priority when the operation is assigned to a device. For example, tf.matmul has both CPU and GPU kernels. On a system with devices CPU:0 and GPU:0, the GPU:0 device will be selected to run tf.matmul unless you explicitly request running it on another device."  

Code below creates tensors and tests them:

**In your terminal, you'll need to tell TensorBoard where your logs are being stored**
Ex. CUS_C02VJ0NQHTDF:~ bblanchard006$ tensorboard --logdir /Users/bblanop/SMU/QTW/logs


'C:/Users/danie/Documents/GitHub/Quantifying-The-World/Case Study 6/logs'

In [ ]:

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Case Study 12

In [ ]:
import os
os.getcwd()
os.chdir('C:\\Users\\danie\\Documents\\GitHub\\Quantifying-The-World\\Case Study 6\\data')

In [ ]:
df = pd.read_csv('HIGGS.csv', header=None)

In [ ]:
# Verify size is as expected
print("{:>25}{:>10,} x {:,}\n".format("Shape of raw data is ", df.shape[0], df.shape[1]))

In [ ]:
df.head()

In [ ]:
df[0].describe()